In [65]:
%reset
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import scipy.stats
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_1samp_test
import gc
import os
import copy
import pickle
from os.path import exists
import mne
import numpy as np
from mne import create_info
from IPython.utils import io
import yasa
import seaborn as sns
import pandas as pd
import matlab
import matlab.engine
import os
import shutil

def overlap(a, b):
    return a[1] >= b[0] and a[0] <= b[1]

gc.collect()

11

In [66]:
pkl_dir = "C:\\Users\\User\\Cloud-Drive\\BigFiles\\OmissionExpOutput\\\imported_eventDetectionChan\\filter0.35"
import_type = "eventDetectionChan"
output_dir_name = 'eventDetection\\done_on_imported'
import_path = f'{pkl_dir}\\{import_type}.pkl'

Batch Nir&Andrillon Detection

In [67]:
electrode_column_name = 'electrode'
header = np.array(['Onset',"Duration","Annotation"])

def detect_ss_AndrillonNir(file_ids,minMax_sd_ver,output_key,electrodes_names):
    andriNir_code_dir = os.getcwd()
    andriNir_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}\\Andrillon_Nir"
    andriNir_aux_output_dir = f"{andriNir_output_dir}\\aux_mats"

    if os.path.exists(andriNir_output_dir):
        shutil.rmtree(andriNir_output_dir)
    os.mkdir(andriNir_output_dir)
    os.mkdir(andriNir_aux_output_dir)

    spindles_output_columns =  ['spindleStartTime', 'spindleEndTime', 'peakTime', 'peakEnergy', 'peakEnergyNorm', 'freqSpindle', 'spindleDuration/SR', 'PowerSP', 'PowerAlpha', 'currentStage']

    for id in file_ids:
        datafile_data = allsubsdata_perFile[id]['data'] ## shape (electrode, time)
        datafile_scoring = allsubsdata_perFile[id]['scoring'] ## shape (time/sampling/30)
        scoring_upsampled = yasa.hypno_upsample_to_data(datafile_scoring, 1/30, datafile_data, sf_data=configu['sample_freq'], verbose=True)
        ss_allElectrodes = pd.DataFrame()

        for electd_i, electrode_name_eventDetect in enumerate(electrodes_names):
            curr_electrode_num = np.where(configu['electrodes_names'] == electrode_name_eventDetect)[0][0]

            ## create aux files to use in MATLAB
            datafile_1elect_eeg = datafile_data[curr_electrode_num,:]
            if 2 not in datafile_scoring or 3 not in datafile_scoring:
                continue
            
            DetectionThreshold = minMax_sd_ver[0]
            RejectThreshold = minMax_sd_ver[1]
            StartEndThreshold = minMax_sd_ver[2]
            mat_to_save = {'datafile_data': datafile_1elect_eeg, 'scoring_upsampled': scoring_upsampled, 'sample_freq': configu['sample_freq'], 'electrode_name':electrode_name_eventDetect, 'DetectionThreshold':DetectionThreshold,'RejectThreshold':RejectThreshold, 'StartEndThreshold':StartEndThreshold}
            scipy.io.savemat(f"{andriNir_aux_output_dir}\\{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat",mat_to_save)

            ## run Andrillon & Nir SS detection over all subjects files
            eng = matlab.engine.start_matlab()
            eng.cd(andriNir_code_dir, nargout=0)
            out = eng.batch_useAndrillonNirSSDetection(andriNir_aux_output_dir, andriNir_output_dir,nargout=0)
            eng.quit()

            ## add the spindles data to the main subject dictionary
            try: 
                spindles_file_name = f"{andriNir_output_dir}\\SS_AN_{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_{electrode_name_eventDetect}AndScoring.mat"
                matlabImport = scipy.io.loadmat(spindles_file_name, simplify_cells=True)
            except Exception: 
                print(f"Error importing spindles sub file at: {allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}")
                continue

            ss = matlabImport['spindles']
            if np.size(np.shape(ss)) == 1: 
                temp = np.zeros((1,np.size(ss)),dtype=object)
                temp[0] = ss
                ss = temp
            if np.size(ss) >0 :
                df = pd.DataFrame(np.double(ss))
                df.columns = spindles_output_columns
                tile_electrode = np.tile(f"{electrode_name_eventDetect}", ss.shape[0])[None].T
                df[electrode_column_name] = tile_electrode
                
                allsubsdata_perFile[id][f'{output_key}@@{electrode_name_eventDetect}'] = df
                if electd_i ==0: ss_allElectrodes =  df
                else:  ss_allElectrodes = pd.concat([ss_allElectrodes,df])

            # sw = matlabImport['allWaves']
            # if np.size(sw) >0 :
            #     tile_electrode = np.tile(f"{electrode_name_eventDetect}", sw.shape[0])[None].T
            #     sw = np.hstack([sw, tile_electrode])
            #     allsubsdata_perFile[id][f'SW_algo-AN@@{electrode_name_eventDetect}'] = np.asarray(sw,dtype=object)  
            #     sw_allElectrodes = np.concatenate((sw_allElectrodes,sw))
    
        if not ss_allElectrodes.empty:
            #print(f"sub:{id}, thresh:{matlabImport['minMax_SD_threshold']},  before {len(ss_allElectrodes)}")
            allsubsdata_perFile[id][output_key]  = ss_allElectrodes
            #allsubsdata_perFile[id][f'SW_algo-AN_all']  = sw_allElectrodes

    minmax_sd_name =  matlabImport['minMax_SD_threshold']
    shutil.rmtree(andriNir_aux_output_dir)
    return minmax_sd_name
def filter_sleep_stages(file_ids,event_key_to_use,after_sleepStage_exclution_key, sleepstages):
    for id in file_ids:
        if event_key_to_use not in allsubsdata_perFile[id]: continue
        event_allElectrodes = allsubsdata_perFile[id][event_key_to_use]
        
        event_filtered_sleepstages = pd.DataFrame([], columns=event_allElectrodes.columns)
        for sleepstage in sleepstages:
            event_filtered_sleepstages = pd.concat([event_filtered_sleepstages , event_allElectrodes.loc[(event_allElectrodes['currentStage'] == sleepstage)]])

        if len(event_filtered_sleepstages) == 0:  print(f"sub:{id} - no events in sleepstages{sleepstages}")
        else: print(f"sub:{id}, before exclude sleep stages:{len(event_allElectrodes)}, after:{len(event_filtered_sleepstages)}")
        
        allsubsdata_perFile[id][after_sleepStage_exclution_key] = event_filtered_sleepstages
def group_spindles(ss_key_to_use, uniqeElctds_ss_key, uniqeElctd_ss_key):
    ## create a spindle array, where each spindle occour once and attribued to the electrode where the spindle was the most powerful
    for id in allsubsdata_perFile:
        if ss_key_to_use not in allsubsdata_perFile[id]:  continue
        filterd_events_allElectrodes = allsubsdata_perFile[id][ss_key_to_use].copy(deep=True)
        
        filterd_events_allElectrodes.sort_values(by=['spindleStartTime'],inplace=True) 
        filterd_events_allElectrodes.reset_index(drop=True, inplace=True)
        deleted = filterd_events_allElectrodes.copy(deep=True)
        filtered = pd.DataFrame([], columns = deleted.columns)
        filtered_moreThan1 = pd.DataFrame([], columns = deleted.columns)
        simultan = pd.DataFrame([], columns = deleted.columns)

        while len(deleted)>0:
            if len(simultan)==0:
                simultan = pd.concat([simultan, deleted.iloc[[0]]])
                deleted.drop(deleted.index[0], axis=0, inplace=True)
                still_overlap = True
            else:
                while still_overlap == True and len(deleted)>0:
                    still_overlap = False
                    simultan.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows

                    ## check now_overlap_all_in_simultan:
                    for index, simultan_row in simultan.iterrows():
                        simultan_0 = simultan_row['spindleStartTime']
                        simultan_1 = simultan_row['spindleEndTime']
                        deleted_0 = deleted.iloc[0]['spindleStartTime']
                        deleted_1 = deleted.iloc[0]['spindleEndTime']
                        if overlap([simultan_0,simultan_1],[deleted_0,deleted_1]):
                            simultan = pd.concat([simultan, deleted.iloc[[0]]])
                            deleted.drop(deleted.index[0], axis=0, inplace=True)
                            still_overlap = True
                            break
                    if still_overlap: continue
                    else:
                        ## check max ps and add to filt
                        # if len(simultan)>1:
                        #     print('hu')
                        simultan.sort_values(by=['PowerSP'],ascending=False,inplace=True)
                        row_df = simultan.iloc[[0]]
                        row_df.iat[0, row_df.columns.get_loc(electrode_column_name)] = np.array(simultan[electrode_column_name])
                        filtered = pd.concat([filtered,row_df])
                        if len(simultan) > 1: filtered_moreThan1 = pd.concat([filtered_moreThan1,row_df])
                        simultan = pd.DataFrame([], columns = deleted.columns)                    
            
        filtered.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows
        filtered_moreThan1.reset_index(drop=True, inplace=True) # make sure indexes pair with number of rows
        allsubsdata_perFile[id][uniqeElctd_ss_key] = filtered   
        allsubsdata_perFile[id][uniqeElctds_ss_key] = filtered_moreThan1   
        print(f"sub:{id}, before filt:{np.shape(allsubsdata_perFile[id][ss_key_to_use])[0]}, after:{np.shape(allsubsdata_perFile[id][uniqeElctd_ss_key])[0]}, after>1: after:{np.shape(allsubsdata_perFile[id][uniqeElctds_ss_key])[0]}")

def edfViewFormat_scoring_dict(score):
    if score == 0:
        return 'W'
    elif score ==1:
        return 'N1'
    elif score ==2:
        return 'N2'
    elif score ==3:
        return 'N3'
    elif score ==4:
        return 'TREM'
    elif score ==5:
        return 'PREM'
    elif score ==6:
        return 'MOVE'
    elif score ==7:
        return 'ARTIFACT'
    else:
        Exception('no such score')
def add_edfViewFormat_scoring(key_edfScoringFormat):
    for id in allsubsdata_perFile:
        curr_file_scoring = allsubsdata_perFile[id]['scoring']
        new_format_score = np.zeros((len(curr_file_scoring),3), dtype=object)
        for ind, score in enumerate(curr_file_scoring):
                new_format_score[ind,:] = [30*ind,30,edfViewFormat_scoring_dict(score)] ## onset (sec), duration, desc

        allsubsdata_perFile[id][key_edfScoringFormat] = new_format_score
def add_edfViewFormat_ss(file_ids,event_key_for_save, SS_efdViewFormat_key):
        for id in file_ids:
            if event_key_for_save not in allsubsdata_perFile[id]:
                    #print(f"no {event_key_for_save} for sub {id}")
                    continue
            ss_df = allsubsdata_perFile[id][event_key_for_save]
            ss_mat_edfFormat = np.zeros((len(ss_df),3), dtype=object)
            startTime_arr = np.array(ss_df['spindleStartTime']) / np.double(configu['sample_freq'])
            endTime_arr = np.array(ss_df['spindleEndTime']) / np.double(configu['sample_freq'])
            duration_arr = endTime_arr - startTime_arr
            electd_arr_per_ss = np.array(ss_df[electrode_column_name])
            desc = [f"SS@@{electd_arr[0]}" for electd_arr in electd_arr_per_ss]
            new_format_ss = np.array([startTime_arr,duration_arr,desc]).T ## onset (sec), duration,desc
            allsubsdata_perFile[id][SS_efdViewFormat_key] = new_format_ss

def save_eventsSepratelyForComparison_edfViewFormat(file_ids,events_types_to_compare,edfViewFormat_output_dir):
    # Save events (scoring and ss) per file, in a format suitable for EDF_viewer
    for id in file_ids:
        for event_type in events_types_to_compare:
            filename = f"{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_events"
            all_events_with_header = np.asarray([header],dtype=object)
            events_type_found = [ v for k, v in allsubsdata_perFile[id].items() if ((event_type in k) and (edfviewFormatSuffix in k))]
            if np.size(events_type_found) == 0: continue
            for events_found in events_type_found:
                    for event_found in events_found:
                            all_events_with_header = np.concatenate((all_events_with_header,[event_found]),dtype=object)
            np.savetxt(f"{edfViewFormat_output_dir}\\{event_type}_{filename}.txt", all_events_with_header, delimiter='\t',fmt='%s')
    
# Save events (scoring and ss) per file, in a format suitable for EDF_viewer
def save_eventsAllTogether_edfViewFormat(events_types_for_save,edfViewFormat_output_dir):
        for id in file_ids:
            filename = f"{allsubsdata_perFile[id]['subnum']}_{allsubsdata_perFile[id]['filenum']}_events"
            all_events_with_header = np.asarray([header],dtype=object)
            for event_type in events_types_for_save:
                events_type_found = [ v for k, v in allsubsdata_perFile[id].items() if (event_type in k) and (edfviewFormatSuffix in k)]
                if np.size(events_type_found) == 0: continue
                for events_found in events_type_found:
                        for event_found in events_found:
                                all_events_with_header = np.concatenate((all_events_with_header,[event_found]),dtype=object)
            np.savetxt(f"{edfViewFormat_output_dir}\\allEvents_{filename}.txt", all_events_with_header, delimiter='\t',fmt='%s')

testType = 1
if(testType==1):
    file_ids = ['38_2']
    minMax_sd_vers = [[4,10,1]]
    electrodes_names_eventDetect = ['Fp2','F3']
elif(testType==2):
    file_ids = ['32_2','38_2']
    minMax_sd_vers = [[3,9,1],[4,10,1]]
    electrodes_names_eventDetect = ['Fp2','F3']
elif(testType==3):
    file_ids = ['32_2','38_2']
    minMax_sd_vers = [[3,9,1],[4,10,1]]
    electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']
else:
    file_ids = ['32_2','38_2']
    electrodes_names_eventDetect = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2']
    minMax_sd_vers = [[3,8,1],[4,8,1],[5,8,1],[3,9,1],[4,9,1],[5,9,1],[3,10,1],[4,10,1],[5,10,1]]
    #file_ids = allsubsdata_perFile.keys()

with open(import_path, "rb") as file:
    [allsubsdata_perFile, configu] = pickle.load(file)
fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

edfViewFormat_events_manual_dir = f'{fig_output_dir}\\EDFViewFormat_events'
edfViewFormat_events_output_dir = f'{fig_output_dir}\\EDFViewFormat_events\\minmax_sd_tests'
if os.path.exists(edfViewFormat_events_output_dir):
    shutil.rmtree(edfViewFormat_events_output_dir)
os.mkdir(edfViewFormat_events_output_dir)

fig_output_dir = f"{configu['outputs_dir_path']}\\{output_dir_name}"
if not os.path.exists(fig_output_dir): os.mkdir(fig_output_dir)

for minMax_sd_ver in minMax_sd_vers:
    with open(import_path, "rb") as file: [allsubsdata_perFile, configu] = pickle.load(file)
    
    all_electodes_ss_key = 'SS_AN'
    minmax_sd_name = detect_ss_AndrillonNir(file_ids,minMax_sd_ver,all_electodes_ss_key,electrodes_names_eventDetect)
    print(minmax_sd_name)

    after_sleepStage_exclution_key = f"{all_electodes_ss_key}_n2n3"
    filter_sleep_stages(file_ids,all_electodes_ss_key,after_sleepStage_exclution_key, [2.0,3.0]) 

    ss_key_to_use = after_sleepStage_exclution_key
    multiElectdPerSS_text = "multiElectd"
    singleElectdPerSS_text = "singleElectd"
    multiElectdPerSS_key = f"{ss_key_to_use}_{multiElectdPerSS_text}"
    singleElectdPerSS_key = f"{ss_key_to_use}_{singleElectdPerSS_text}"
    group_spindles(ss_key_to_use,multiElectdPerSS_key, singleElectdPerSS_key)

    edfviewFormatSuffix = 'efdViewFormat'
    SS_multiElectdPerSS_efdViewFormat_key = f'{multiElectdPerSS_key}_{edfviewFormatSuffix}'
    SS_singleElectdPerSS_efdViewFormat_key = f'{singleElectdPerSS_key}_{edfviewFormatSuffix}'
    add_edfViewFormat_ss(file_ids,multiElectdPerSS_key,SS_multiElectdPerSS_efdViewFormat_key)
    add_edfViewFormat_ss(file_ids,singleElectdPerSS_key,SS_singleElectdPerSS_efdViewFormat_key)

    events_types_to_compare = [multiElectdPerSS_text,singleElectdPerSS_text]    
    edfViewFormat_eventsTest_output_dir = f"{edfViewFormat_events_output_dir}\\{minMax_sd_ver}"
    if os.path.exists(edfViewFormat_eventsTest_output_dir):  shutil.rmtree(edfViewFormat_eventsTest_output_dir)
    os.mkdir(edfViewFormat_eventsTest_output_dir)
    save_eventsSepratelyForComparison_edfViewFormat(file_ids,events_types_to_compare,edfViewFormat_eventsTest_output_dir)

    events_types_to_save = ['scoring', multiElectdPerSS_text,singleElectdPerSS_text]    
    save_eventsAllTogether_edfViewFormat(events_types_to_save,edfViewFormat_eventsTest_output_dir)

05-פבר-23 20:38:03 | WARNING | Hypnogram is SHORTER than data by 23.01 seconds. Padding hypnogram with last value to match data.size.


[ 4 10  1]
sub:38_2, before exclude sleep stages:537, after:375
sub:38_2, before filt:375, after:278, after>1: after:95


In [68]:
from IPython.core.display import HTML
#display(HTML(allsubsdata_perFile[file_ids[0]][singleElectdPerSS_key].to_html()))

In [69]:
def preform_minmaxSD_comparison(events_types_to_compare,file_ids,minMax_sd_vers):
    all_comparisons = pd.DataFrame(columns=['id','eventType','minmax_ver','Positive','TruePos','FalsePos', 'hitRate','falseDiscoveryRate'])
    for id in file_ids:
        for ver in minMax_sd_vers:
            for event_type in events_types_to_compare:
                dir_auto_ = f"{edfViewFormat_events_output_dir}\\{ver}"
                manual_detection_filename = f"{edfViewFormat_events_manual_dir}\\sub{id.split('_')[0]}_sleep{id.split('_')[1]}_imported_eventDetectionChan_annotations.txt"
                auto_detection_filename = f"{dir_auto_}\\{event_type}_{id}_events.txt"
                ## get the array of before manual scanning
                if os.path.exists(auto_detection_filename) and os.path.exists(manual_detection_filename):
                    all_ss_auto = np.loadtxt(auto_detection_filename, delimiter="\t",dtype='object')
                    all_ss_auto = np.delete(all_ss_auto, np.where(all_ss_auto[:, 2] == "Annotation")[0], axis=0)
                    all_ss_auto[:,[0,1]] = [np.double(x) for x in all_ss_auto[:,[0,1]]]
                    ss_ind = np.array([],dtype=int)
                    for ind_i, desc in enumerate(all_ss_auto[:,2]):
                        if ("SS" in desc) or ("ss" in desc):
                            ss_ind = np.append(ss_ind, ind_i)
                    all_ss_auto = all_ss_auto[ss_ind,:]

                    ## get the array of after manual scanning
                    all_ss_manu = np.loadtxt(manual_detection_filename, delimiter="\t",dtype='object')
                    all_ss_manu = np.delete(all_ss_manu, np.where(all_ss_manu[:, 2] == "Annotation")[0], axis=0)
                    all_ss_manu[:,[0,1]] = [np.double(x) for x in all_ss_manu[:,[0,1]]]
                    ss_ind = np.array([],dtype=int)
                    for ind_i, desc in enumerate(all_ss_manu[:,2]):
                        if "SS" in desc:
                            ss_ind = np.append(ss_ind, ind_i)
                    all_ss_manu = all_ss_manu[ss_ind,:]

                    ## compare to find rate od TP and FP
                    TP = 0
                    FP = 0
                    for ss_auto in all_ss_auto:
                        found = False
                        for ss_manu in all_ss_manu:
                            if overlap([ss_auto[0],ss_auto[0]+ss_auto[1]],[ss_manu[0],ss_manu[0]+ss_manu[1]]):
                                TP +=1
                                found = True
                                break
                        if found == False:
                            FP +=1

                    # FN = 0
                    # for ss_manu in all_ss_manu:
                    #     found = False
                    #     for ss_auto in all_ss_auto:
                    #         if overlap([ss_auto[0],ss_auto[0]+ss_auto[1]],[ss_manu[0],ss_manu[0]+ss_manu[1]]):
                    #             found = True
                    #             break
                    #     if found == False:
                    #         FN +=1

                    Positive = np.shape(all_ss_manu)[0]   
                    hitRate = TP/Positive
                    #missRate = FN/Positive
                    falseDiscoveryRate = FP/(FP+TP)
                    
                    comparison = [id,event_type, ver, Positive, TP, FP,hitRate, falseDiscoveryRate]
                    all_comparisons.loc[len(all_comparisons)] = comparison

    all_comparisons = all_comparisons.sort_values('minmax_ver')
    all_comparisons = all_comparisons.sort_values('id')
    return all_comparisons
    
events_types_to_compare = [multiElectdPerSS_text,singleElectdPerSS_text]
all_comp = preform_minmaxSD_comparison(events_types_to_compare,file_ids,minMax_sd_vers)
all_comp.to_csv('ss_comparisons.csv',index=False)
all_comp

,id,eventType,minmax_ver,Positive,TruePos,FalsePos,hitRate,falseDiscoveryRate
0,38_2,multiElectd,"[4, 10, 1]",448,89,6,0.198661,0.063158
1,38_2,singleElectd,"[4, 10, 1]",448,254,24,0.566964,0.086331


In [70]:
## how to df 

## add row to df and empty df
x = pd.DataFrame([], columns = deleted.columns)
y= deleted.iloc[[0]]
y2= deleted.iloc[[1]]
z = pd.concat([x,y])
zz = pd.concat([z,y2])
display(zz)

## delete first row
# zz.drop(zz.index[0], axis=0, inplace=True) 
# display(zz)

## get value at i,j
print(deleted.iloc[0]['spindleStartTime'])

## get column as array
arr = np.array(zz['electrode'])
print(arr)

## replace inplace value in i,j to array:
zz.iat[1, zz.columns.get_loc(electrode_column_name)] = arr
zz

NameError: name 'deleted' is not defined